In [1]:
import numpy as np
import cv2

In [2]:
#Setting parameters for Shi-Tomasi Corner Detection
corner_track_params = dict(maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

In [3]:
#Setting parameters for Lucas-Kanade Method
lk_params = dict(winSize = (15,15), maxLevel = 2, #maxlevel = 0 provides the original image, hence these levels provide optical flow at various resolutions of the image
               criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [ ]:
cap = cv2.VideoCapture(0)

# Check if the video capture is opened
if not cap.isOpened():
    print("Error1: Could not open video capture")
    exit()
cv2.waitKey(1000)

#Read the first frame
ret, prev_frame = cap.read()
if not ret or prev_frame is None:
    print("Error2: Could not read first frame")
    cap.release()
    exit()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

#Points to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    
    if not ret or frame is None:
        print("Error3: Could not read frame")
        break
        
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None,**lk_params) #PyrLK is Pyramid LK

    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1] #it will be 1 if the flow for the corresponding features has been found

    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        x_new, y_new = new.ravel().astype(int)
        x_prev, y_prev = prev.ravel().astype(int)

        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3) #color is green and thickness is 3

        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)

    img = cv2.add(frame,mask)
    
    cv2.imshow('tracking',img)

    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    #Update previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release

In [ ]:
#Dense Optical Flow in OpenCV

cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()
if not ret:
    print("Error1: Could not read the first frame.")
    cap.release()
    exit()

prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

#Initializing Hue, Saturation and Value
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    if not ret:
        print("Error2: Could not read frame.")
        break

    nextImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    
    #Calculating the dense optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0)

    #Computing the magnitude and angle of the 2D vectors
    mag, ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees = True)

    hsv_mask[:,:,0] = ang/2 #setting hue
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX) #setting value ccording to the optical flow magnitude

    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame',bgr)

    k = cv2.waitKey(10)& 0xFF
    if k == 27:
        break

    #Update the previous frame
    prvsTmg = nextImg

cap.release()
cv2.destroyAllWindows()

#If you move up the color will change to purple , if you move down it will change to green
#Similarly if you move left it will change to blue if you move right it will change to red